#### M5 Forecasting
I thought it would be interesting to work on a forecasting project. Sales Forecasting is a very well-understood area and makes for good conversation during interviews. All the data was downloaded and uploaded to AWS S3. The goal of this competition is to estimate the unit sales of Walmart retail goods

#### Importing libraries

In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
import seaborn as sns
import boto3
from io import StringIO


Matplotlib is building the font cache; this may take a moment.


#### Reading the raw data from AWS S3

In [2]:
# S3 bucket and folder
bucket_name = "walmart-forecast-data"
prefix = "data/"  # Folder inside the bucket

# List of all the files
files = ["calendar.csv", "sales_train_evaluation.csv", "sales_train_validation.csv",
         "sample_submission.csv", "sell_prices.csv"]

# Initializing the S3 client
s3 = boto3.client("s3")

# Function to read CSV from S3
def read_s3_csv(file_name):
    obj = s3.get_object(Bucket=bucket_name, Key=prefix + file_name)
    return pd.read_csv(StringIO(obj["Body"].read().decode("utf-8")))

# Load all files into Pandas DataFrames
calendar = read_s3_csv("calendar.csv")
sales_evaluation = read_s3_csv("sales_train_evaluation.csv")
sales_validation = read_s3_csv("sales_train_validation.csv")
prices = read_s3_csv("sell_prices.csv")


print("Calendar:", calendar.shape)
print("Sales Train Evaluation:", sales_evaluation.shape)
print("Sell Prices:", prices.shape)
print("Sales Train Validation", sales_validation.shape)

Calendar: (1969, 14)
Sales Train Evaluation: (30490, 1947)
Sell Prices: (6841121, 4)
Sales Train Validation (30490, 1919)


In [3]:
calendar

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1964,2016-06-15,11620,Wednesday,5,6,2016,d_1965,NaN,NaN,NaN,NaN,0,1,1
1965,2016-06-16,11620,Thursday,6,6,2016,d_1966,NaN,NaN,NaN,NaN,0,0,0
1966,2016-06-17,11620,Friday,7,6,2016,d_1967,NaN,NaN,NaN,NaN,0,0,0
1967,2016-06-18,11621,Saturday,1,6,2016,d_1968,NaN,NaN,NaN,NaN,0,0,0


In [4]:
sales_evaluation

id        item_id    dept_id   cat_id  \
0      HOBBIES_1_001_CA_1_evaluation  HOBBIES_1_001  HOBBIES_1  HOBBIES   
1      HOBBIES_1_002_CA_1_evaluation  HOBBIES_1_002  HOBBIES_1  HOBBIES   
2      HOBBIES_1_003_CA_1_evaluation  HOBBIES_1_003  HOBBIES_1  HOBBIES   
3      HOBBIES_1_004_CA_1_evaluation  HOBBIES_1_004  HOBBIES_1  HOBBIES   
4      HOBBIES_1_005_CA_1_evaluation  HOBBIES_1_005  HOBBIES_1  HOBBIES   
...                              ...            ...        ...      ...   
30485    FOODS_3_823_WI_3_evaluation    FOODS_3_823    FOODS_3    FOODS   
30486    FOODS_3_824_WI_3_evaluation    FOODS_3_824    FOODS_3    FOODS   
30487    FOODS_3_825_WI_3_evaluation    FOODS_3_825    FOODS_3    FOODS   
30488    FOODS_3_826_WI_3_evaluation    FOODS_3_826    FOODS_3    FOODS   
30489    FOODS_3_827_WI_3_evaluation    FOODS_3_827    FOODS_3    FOODS   

      store_id state_id  d_1  d_2  d_3  d_4  d_5  d_6  d_7  d_8  d_9  d_10  \
0         CA_1       CA    0    0    0    0    0    0    0    0    0     0   
1         CA_1       CA    0    0    0    0    0    0    0    0    0     0   
2         CA_1       CA    0    0    0    0    0    0    0    0    0     0   
3         CA_1       CA    0    0    0    0    0    0    0    0    0     0   
4         CA_1       CA    0    0    0    0    0    0    0    0    0     0   
...        ...      ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   ...   
30485     WI_3       WI    0    0    2    2    0    3    1    4    1     0   
30486     WI_3       WI    0    0    0    0    0    5    0    1    1     3   
30487     WI_3       WI    0    6    0    2    2    4    1    8    5     2   
30488     WI_3       WI    0    0    0    0    0    0    0    0    0     0   
30489     WI_3       WI    0    0    0    0    0    0    0    0    0     0   

       d_11  d_12  d_13  d_14  d_15  d_16  d_17  d_18  d_19  d_20  d_21  d_22  \
0         0     0     0     0     0     0     0     0     0     0     0     0   
1         0     0     0     0     0     0     0     0     0     0     0     0   
2         0     0     0     0     0     0     0     0     0     0     0     0   
3         0     0     0     0     0     0     0     0     0     0     0     0   
4         0     0     0     0     0     0     0     0     0     0     0     0   
...     ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   
30485     0     3     4     4     0     0     1     0     1     1     7     7   
30486     1     1     0     4     2     0     1     2     1     1     0     0   
30487     7     5     3     5    20     8    10     3     3     4     7     2   
30488     0     0     0     0     0     0     0     0     0     0     0     0   
30489     0     0     0     0     0     0     0     0     0     0     0     0   

       d_23  d_24  d_25  d_26  d_27  d_28  d_29  d_30  d_31  d_32  d_33  d_34  \
0         0     0     0     0     0     0     0     0     0     0     0     0   
1         0     0     0     0     0     0     0     0     0     0     0     0   
2         0     0     0     0     0     0     0     0     0     0     0     0   
3         0     0     0     0     0     0     0     0     0     0     0     0   
4         0     0     0     0     0     0     0     0     0     0     0     0   
...     ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   
30485     3     6     3     3     7    12     4     2     7     5    12     5   
30486     0     0     3     1     1     1     2     0     1     0     1     1   
30487     3     5     6     3     4     1     2     5     1     2     2     2   
30488     0     0     0     0     0     0     0     0     0     0     0     0   
30489     0     0     0     0     0     0     0     0     0     0     0     0   

       d_35  d_36  d_37  d_38  d_39  d_40  d_41  d_42  d_43  d_44  d_45  d_46  \
0         0     0     0     0     0     0     0     0     0     0     0     0   
1         0     0     0     0     0     0     0     0     0     0     0     0   
2      

In [5]:
sales_validation

id        item_id    dept_id   cat_id  \
0      HOBBIES_1_001_CA_1_validation  HOBBIES_1_001  HOBBIES_1  HOBBIES   
1      HOBBIES_1_002_CA_1_validation  HOBBIES_1_002  HOBBIES_1  HOBBIES   
2      HOBBIES_1_003_CA_1_validation  HOBBIES_1_003  HOBBIES_1  HOBBIES   
3      HOBBIES_1_004_CA_1_validation  HOBBIES_1_004  HOBBIES_1  HOBBIES   
4      HOBBIES_1_005_CA_1_validation  HOBBIES_1_005  HOBBIES_1  HOBBIES   
...                              ...            ...        ...      ...   
30485    FOODS_3_823_WI_3_validation    FOODS_3_823    FOODS_3    FOODS   
30486    FOODS_3_824_WI_3_validation    FOODS_3_824    FOODS_3    FOODS   
30487    FOODS_3_825_WI_3_validation    FOODS_3_825    FOODS_3    FOODS   
30488    FOODS_3_826_WI_3_validation    FOODS_3_826    FOODS_3    FOODS   
30489    FOODS_3_827_WI_3_validation    FOODS_3_827    FOODS_3    FOODS   

      store_id state_id  d_1  d_2  d_3  d_4  d_5  d_6  d_7  d_8  d_9  d_10  \
0         CA_1       CA    0    0    0    0    0    0    0    0    0     0   
1         CA_1       CA    0    0    0    0    0    0    0    0    0     0   
2         CA_1       CA    0    0    0    0    0    0    0    0    0     0   
3         CA_1       CA    0    0    0    0    0    0    0    0    0     0   
4         CA_1       CA    0    0    0    0    0    0    0    0    0     0   
...        ...      ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   ...   
30485     WI_3       WI    0    0    2    2    0    3    1    4    1     0   
30486     WI_3       WI    0    0    0    0    0    5    0    1    1     3   
30487     WI_3       WI    0    6    0    2    2    4    1    8    5     2   
30488     WI_3       WI    0    0    0    0    0    0    0    0    0     0   
30489     WI_3       WI    0    0    0    0    0    0    0    0    0     0   

       d_11  d_12  d_13  d_14  d_15  d_16  d_17  d_18  d_19  d_20  d_21  d_22  \
0         0     0     0     0     0     0     0     0     0     0     0     0   
1         0     0     0     0     0     0     0     0     0     0     0     0   
2         0     0     0     0     0     0     0     0     0     0     0     0   
3         0     0     0     0     0     0     0     0     0     0     0     0   
4         0     0     0     0     0     0     0     0     0     0     0     0   
...     ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   
30485     0     3     4     4     0     0     1     0     1     1     7     7   
30486     1     1     0     4     2     0     1     2     1     1     0     0   
30487     7     5     3     5    20     8    10     3     3     4     7     2   
30488     0     0     0     0     0     0     0     0     0     0     0     0   
30489     0     0     0     0     0     0     0     0     0     0     0     0   

       d_23  d_24  d_25  d_26  d_27  d_28  d_29  d_30  d_31  d_32  d_33  d_34  \
0         0     0     0     0     0     0     0     0     0     0     0     0   
1         0     0     0     0     0     0     0     0     0     0     0     0   
2         0     0     0     0     0     0     0     0     0     0     0     0   
3         0     0     0     0     0     0     0     0     0     0     0     0   
4         0     0     0     0     0     0     0     0     0     0     0     0   
...     ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   
30485     3     6     3     3     7    12     4     2     7     5    12     5   
30486     0     0     3     1     1     1     2     0     1     0     1     1   
30487     3     5     6     3     4     1     2     5     1     2     2     2   
30488     0     0     0     0     0     0     0     0     0     0     0     0   
30489     0     0     0     0     0     0     0     0     0     0     0     0   

       d_35  d_36  d_37  d_38  d_39  d_40  d_41  d_42  d_43  d_44  d_45  d_46  \
0         0     0     0     0     0     0     0     0     0     0     0     0   
1         0     0     0     0     0     0     0     0     0     0     0     0   
2      

In [6]:
prices

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26
...,...,...,...,...
6841116,WI_3,FOODS_3_827,11617,1.00
6841117,WI_3,FOODS_3_827,11618,1.00
6841118,WI_3,FOODS_3_827,11619,1.00
6841119,WI_3,FOODS_3_827,11620,1.00


In [7]:
for d in range(1942, 1970):
    col = f'd_{d}'
    sales_evaluation[col] = 0  # Placeholder for future sales
    sales_evaluation[col] = sales_evaluation[col].astype(np.int16)

for d in range(1914, 1942):  # Validation set range
    col = f'd_{d}'
    if col not in sales_validation.columns:  
        sales_validation[col] = 0
        sales_validation[col] = sales_validation[col].astype(np.int16)


In [8]:
sales_evaluation

id        item_id    dept_id   cat_id  \
0      HOBBIES_1_001_CA_1_evaluation  HOBBIES_1_001  HOBBIES_1  HOBBIES   
1      HOBBIES_1_002_CA_1_evaluation  HOBBIES_1_002  HOBBIES_1  HOBBIES   
2      HOBBIES_1_003_CA_1_evaluation  HOBBIES_1_003  HOBBIES_1  HOBBIES   
3      HOBBIES_1_004_CA_1_evaluation  HOBBIES_1_004  HOBBIES_1  HOBBIES   
4      HOBBIES_1_005_CA_1_evaluation  HOBBIES_1_005  HOBBIES_1  HOBBIES   
...                              ...            ...        ...      ...   
30485    FOODS_3_823_WI_3_evaluation    FOODS_3_823    FOODS_3    FOODS   
30486    FOODS_3_824_WI_3_evaluation    FOODS_3_824    FOODS_3    FOODS   
30487    FOODS_3_825_WI_3_evaluation    FOODS_3_825    FOODS_3    FOODS   
30488    FOODS_3_826_WI_3_evaluation    FOODS_3_826    FOODS_3    FOODS   
30489    FOODS_3_827_WI_3_evaluation    FOODS_3_827    FOODS_3    FOODS   

      store_id state_id  d_1  d_2  d_3  d_4  d_5  d_6  d_7  d_8  d_9  d_10  \
0         CA_1       CA    0    0    0    0    0    0    0    0    0     0   
1         CA_1       CA    0    0    0    0    0    0    0    0    0     0   
2         CA_1       CA    0    0    0    0    0    0    0    0    0     0   
3         CA_1       CA    0    0    0    0    0    0    0    0    0     0   
4         CA_1       CA    0    0    0    0    0    0    0    0    0     0   
...        ...      ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   ...   
30485     WI_3       WI    0    0    2    2    0    3    1    4    1     0   
30486     WI_3       WI    0    0    0    0    0    5    0    1    1     3   
30487     WI_3       WI    0    6    0    2    2    4    1    8    5     2   
30488     WI_3       WI    0    0    0    0    0    0    0    0    0     0   
30489     WI_3       WI    0    0    0    0    0    0    0    0    0     0   

       d_11  d_12  d_13  d_14  d_15  d_16  d_17  d_18  d_19  d_20  d_21  d_22  \
0         0     0     0     0     0     0     0     0     0     0     0     0   
1         0     0     0     0     0     0     0     0     0     0     0     0   
2         0     0     0     0     0     0     0     0     0     0     0     0   
3         0     0     0     0     0     0     0     0     0     0     0     0   
4         0     0     0     0     0     0     0     0     0     0     0     0   
...     ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   
30485     0     3     4     4     0     0     1     0     1     1     7     7   
30486     1     1     0     4     2     0     1     2     1     1     0     0   
30487     7     5     3     5    20     8    10     3     3     4     7     2   
30488     0     0     0     0     0     0     0     0     0     0     0     0   
30489     0     0     0     0     0     0     0     0     0     0     0     0   

       d_23  d_24  d_25  d_26  d_27  d_28  d_29  d_30  d_31  d_32  d_33  d_34  \
0         0     0     0     0     0     0     0     0     0     0     0     0   
1         0     0     0     0     0     0     0     0     0     0     0     0   
2         0     0     0     0     0     0     0     0     0     0     0     0   
3         0     0     0     0     0     0     0     0     0     0     0     0   
4         0     0     0     0     0     0     0     0     0     0     0     0   
...     ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   
30485     3     6     3     3     7    12     4     2     7     5    12     5   
30486     0     0     3     1     1     1     2     0     1     0     1     1   
30487     3     5     6     3     4     1     2     5     1     2     2     2   
30488     0     0     0     0     0     0     0     0     0     0     0     0   
30489     0     0     0     0     0     0     0     0     0     0     0     0   

       d_35  d_36  d_37  d_38  d_39  d_40  d_41  d_42  d_43  d_44  d_45  d_46  \
0         0     0     0     0     0     0     0     0     0     0     0     0   
1         0     0     0     0     0     0     0     0     0     0     0     0   
2      

In [9]:
sales_validation

id        item_id    dept_id   cat_id  \
0      HOBBIES_1_001_CA_1_validation  HOBBIES_1_001  HOBBIES_1  HOBBIES   
1      HOBBIES_1_002_CA_1_validation  HOBBIES_1_002  HOBBIES_1  HOBBIES   
2      HOBBIES_1_003_CA_1_validation  HOBBIES_1_003  HOBBIES_1  HOBBIES   
3      HOBBIES_1_004_CA_1_validation  HOBBIES_1_004  HOBBIES_1  HOBBIES   
4      HOBBIES_1_005_CA_1_validation  HOBBIES_1_005  HOBBIES_1  HOBBIES   
...                              ...            ...        ...      ...   
30485    FOODS_3_823_WI_3_validation    FOODS_3_823    FOODS_3    FOODS   
30486    FOODS_3_824_WI_3_validation    FOODS_3_824    FOODS_3    FOODS   
30487    FOODS_3_825_WI_3_validation    FOODS_3_825    FOODS_3    FOODS   
30488    FOODS_3_826_WI_3_validation    FOODS_3_826    FOODS_3    FOODS   
30489    FOODS_3_827_WI_3_validation    FOODS_3_827    FOODS_3    FOODS   

      store_id state_id  d_1  d_2  d_3  d_4  d_5  d_6  d_7  d_8  d_9  d_10  \
0         CA_1       CA    0    0    0    0    0    0    0    0    0     0   
1         CA_1       CA    0    0    0    0    0    0    0    0    0     0   
2         CA_1       CA    0    0    0    0    0    0    0    0    0     0   
3         CA_1       CA    0    0    0    0    0    0    0    0    0     0   
4         CA_1       CA    0    0    0    0    0    0    0    0    0     0   
...        ...      ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   ...   
30485     WI_3       WI    0    0    2    2    0    3    1    4    1     0   
30486     WI_3       WI    0    0    0    0    0    5    0    1    1     3   
30487     WI_3       WI    0    6    0    2    2    4    1    8    5     2   
30488     WI_3       WI    0    0    0    0    0    0    0    0    0     0   
30489     WI_3       WI    0    0    0    0    0    0    0    0    0     0   

       d_11  d_12  d_13  d_14  d_15  d_16  d_17  d_18  d_19  d_20  d_21  d_22  \
0         0     0     0     0     0     0     0     0     0     0     0     0   
1         0     0     0     0     0     0     0     0     0     0     0     0   
2         0     0     0     0     0     0     0     0     0     0     0     0   
3         0     0     0     0     0     0     0     0     0     0     0     0   
4         0     0     0     0     0     0     0     0     0     0     0     0   
...     ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   
30485     0     3     4     4     0     0     1     0     1     1     7     7   
30486     1     1     0     4     2     0     1     2     1     1     0     0   
30487     7     5     3     5    20     8    10     3     3     4     7     2   
30488     0     0     0     0     0     0     0     0     0     0     0     0   
30489     0     0     0     0     0     0     0     0     0     0     0     0   

       d_23  d_24  d_25  d_26  d_27  d_28  d_29  d_30  d_31  d_32  d_33  d_34  \
0         0     0     0     0     0     0     0     0     0     0     0     0   
1         0     0     0     0     0     0     0     0     0     0     0     0   
2         0     0     0     0     0     0     0     0     0     0     0     0   
3         0     0     0     0     0     0     0     0     0     0     0     0   
4         0     0     0     0     0     0     0     0     0     0     0     0   
...     ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   
30485     3     6     3     3     7    12     4     2     7     5    12     5   
30486     0     0     3     1     1     1     2     0     1     0     1     1   
30487     3     5     6     3     4     1     2     5     1     2     2     2   
30488     0     0     0     0     0     0     0     0     0     0     0     0   
30489     0     0     0     0     0     0     0     0     0     0     0     0   

       d_35  d_36  d_37  d_38  d_39  d_40  d_41  d_42  d_43  d_44  d_45  d_46  \
0         0     0     0     0     0     0     0     0     0     0     0     0   
1         0     0     0     0     0     0     0     0     0     0     0     0   
2      

In [10]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':  # Integer types
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                else:
                    df[col] = df[col].astype(np.int64)
            else:  # Float types
                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose:
        print(f'Mem. usage decreased to {end_mem:.2f} Mb ({100 * (start_mem - end_mem) / start_mem:.1f}% reduction)')
    
    return df


In [11]:
sales_evaluation = reduce_mem_usage(sales_evaluation)
sales_validation = reduce_mem_usage(sales_validation)
calendar = reduce_mem_usage(calendar)
prices = reduce_mem_usage(prices)

Mem. usage decreased to 96.94 Mb (78.7% reduction)
Mem. usage decreased to 95.81 Mb (78.6% reduction)
Mem. usage decreased to 0.12 Mb (41.9% reduction)
Mem. usage decreased to 143.53 Mb (31.2% reduction)


#### Melting

In [12]:
sales_evaluation = pd.melt(sales_evaluation, id_vars = ['id','item_id','dept_id','cat_id','store_id','state_id'], var_name = 'd', value_name = 'sales')
sales_validation = pd.melt(sales_validation, id_vars = ['id','item_id','dept_id','cat_id','store_id','state_id'], var_name = 'd', value_name = 'sales')

In [13]:
sales_evaluation

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
...,...,...,...,...,...,...,...,...
60034805,FOODS_3_823_WI_3_evaluation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,d_1969,0
60034806,FOODS_3_824_WI_3_evaluation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,d_1969,0
60034807,FOODS_3_825_WI_3_evaluation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,d_1969,0
60034808,FOODS_3_826_WI_3_evaluation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,d_1969,0


In [14]:
sales_validation

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
...,...,...,...,...,...,...,...,...
59181085,FOODS_3_823_WI_3_validation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,d_1941,0
59181086,FOODS_3_824_WI_3_validation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,d_1941,0
59181087,FOODS_3_825_WI_3_validation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,d_1941,0
59181088,FOODS_3_826_WI_3_validation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,d_1941,0


In [15]:
# Merge with calendar data on 'd' column
evaluation_df = sales_evaluation.merge(calendar, on='d', how='left')
validation_df = sales_validation.merge(calendar, on='d', how='left')


In [16]:
# Merge with sell_prices using (store_id, item_id, wm_yr_wk)
evaluation_df = evaluation_df.merge(prices, on=['store_id', 'item_id', 'wm_yr_wk'], how='left')
validation_df = validation_df.merge(prices, on=['store_id', 'item_id', 'wm_yr_wk'], how='left')

In [17]:
evaluation_df

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60034805,FOODS_3_823_WI_3_evaluation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,d_1969,0,2016-06-19,11621,Sunday,2,6,2016,NBAFinalsEnd,Sporting,Father's day,Cultural,0,0,0,2.98
60034806,FOODS_3_824_WI_3_evaluation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,d_1969,0,2016-06-19,11621,Sunday,2,6,2016,NBAFinalsEnd,Sporting,Father's day,Cultural,0,0,0,2.48
60034807,FOODS_3_825_WI_3_evaluation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,d_1969,0,2016-06-19,11621,Sunday,2,6,2016,NBAFinalsEnd,Sporting,Father's day,Cultural,0,0,0,3.98
60034808,FOODS_3_826_WI_3_evaluation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,d_1969,0,2016-06-19,11621,Sunday,2,6,2016,NBAFinalsEnd,Sporting,Father's day,Cultural,0,0,0,1.28


In [18]:
validation_df

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59181085,FOODS_3_823_WI_3_validation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,d_1941,0,2016-05-22,11617,Sunday,2,5,2016,NaN,NaN,NaN,NaN,0,0,0,2.98
59181086,FOODS_3_824_WI_3_validation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,d_1941,0,2016-05-22,11617,Sunday,2,5,2016,NaN,NaN,NaN,NaN,0,0,0,2.48
59181087,FOODS_3_825_WI_3_validation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,d_1941,0,2016-05-22,11617,Sunday,2,5,2016,NaN,NaN,NaN,NaN,0,0,0,3.98
59181088,FOODS_3_826_WI_3_validation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,d_1941,0,2016-05-22,11617,Sunday,2,5,2016,NaN,NaN,NaN,NaN,0,0,0,1.28


In [19]:
evaluation_df.isnull().sum()

id                     0
item_id                0
dept_id                0
cat_id                 0
store_id               0
state_id               0
d                      0
sales                  0
date                   0
wm_yr_wk               0
weekday                0
wday                   0
month                  0
year                   0
event_name_1    55095430
event_type_1    55095430
event_name_2    59882360
event_type_2    59882360
snap_CA                0
snap_TX                0
snap_WI                0
sell_price      12299413
dtype: int64

In [20]:
validation_df.isnull().sum()

id                     0
item_id                0
dept_id                0
cat_id                 0
store_id               0
state_id               0
d                      0
sales                  0
date                   0
wm_yr_wk               0
weekday                0
wday                   0
month                  0
year                   0
event_name_1    54363670
event_type_1    54363670
event_name_2    59059130
event_type_2    59059130
snap_CA                0
snap_TX                0
snap_WI                0
sell_price      12299413
dtype: int64

In [21]:
# # Define categorical columns
# categorical_cols = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 
#                     'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 
#                     'snap_CA', 'snap_TX', 'snap_WI']

# Fill missing values in event-related columns with "No Event"
event_cols = ['event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']

evaluation_df[event_cols] = evaluation_df[event_cols].fillna('No Event')
validation_df[event_cols] = validation_df[event_cols].fillna('No Event')

# # Convert columns to category dtype
# evaluation_df[categorical_cols] = evaluation_df[categorical_cols].astype('category')
# validation_df[categorical_cols] = validation_df[categorical_cols].astype('category')


In [22]:
evaluation_df['sell_price'] = evaluation_df.groupby(['store_id', 'item_id'])['sell_price'].ffill().bfill()
validation_df['sell_price'] = validation_df.groupby(['store_id', 'item_id'])['sell_price'].ffill().bfill()


In [23]:
evaluation_df.isnull().sum()

id              0
item_id         0
dept_id         0
cat_id          0
store_id        0
state_id        0
d               0
sales           0
date            0
wm_yr_wk        0
weekday         0
wday            0
month           0
year            0
event_name_1    0
event_type_1    0
event_name_2    0
event_type_2    0
snap_CA         0
snap_TX         0
snap_WI         0
sell_price      0
dtype: int64

In [24]:
validation_df.isnull().sum()

id              0
item_id         0
dept_id         0
cat_id          0
store_id        0
state_id        0
d               0
sales           0
date            0
wm_yr_wk        0
weekday         0
wday            0
month           0
year            0
event_name_1    0
event_type_1    0
event_name_2    0
event_type_2    0
snap_CA         0
snap_TX         0
snap_WI         0
sell_price      0
dtype: int64

In [25]:
# # Identify categorical columns
# categorical_cols = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 
#                     'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']

# for col in categorical_cols:
#     evaluation_df[col] = evaluation_df[col].astype('category').cat.codes
#     validation_df[col] = validation_df[col].astype('category').cat.codes


In [26]:
evaluation_df.dtypes

id               object
item_id          object
dept_id          object
cat_id           object
store_id         object
state_id         object
d                object
sales             int16
date             object
wm_yr_wk          int16
weekday          object
wday               int8
month              int8
year              int16
event_name_1     object
event_type_1     object
event_name_2     object
event_type_2     object
snap_CA            int8
snap_TX            int8
snap_WI            int8
sell_price      float32
dtype: object

In [27]:
validation_df.dtypes

id               object
item_id          object
dept_id          object
cat_id           object
store_id         object
state_id         object
d                object
sales             int16
date             object
wm_yr_wk          int16
weekday          object
wday               int8
month              int8
year              int16
event_name_1     object
event_type_1     object
event_name_2     object
event_type_2     object
snap_CA            int8
snap_TX            int8
snap_WI            int8
sell_price      float32
dtype: object

In [28]:
evaluation_df['date'] = pd.to_datetime(evaluation_df['date'])
validation_df['date'] = pd.to_datetime(validation_df['date'])

In [29]:
evaluation_df

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,1,1,2011,No Event,No Event,No Event,No Event,0,0,0,0.46
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,1,1,2011,No Event,No Event,No Event,No Event,0,0,0,0.46
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,1,1,2011,No Event,No Event,No Event,No Event,0,0,0,0.46
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,1,1,2011,No Event,No Event,No Event,No Event,0,0,0,0.46
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,1,1,2011,No Event,No Event,No Event,No Event,0,0,0,0.46
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60034805,FOODS_3_823_WI_3_evaluation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,d_1969,0,2016-06-19,11621,Sunday,2,6,2016,NBAFinalsEnd,Sporting,Father's day,Cultural,0,0,0,2.98
60034806,FOODS_3_824_WI_3_evaluation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,d_1969,0,2016-06-19,11621,Sunday,2,6,2016,NBAFinalsEnd,Sporting,Father's day,Cultural,0,0,0,2.48
60034807,FOODS_3_825_WI_3_evaluation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,d_1969,0,2016-06-19,11621,Sunday,2,6,2016,NBAFinalsEnd,Sporting,Father's day,Cultural,0,0,0,3.98
60034808,FOODS_3_826_WI_3_evaluation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,d_1969,0,2016-06-19,11621,Sunday,2,6,2016,NBAFinalsEnd,Sporting,Father's day,Cultural,0,0,0,1.28


In [30]:
evaluation_df['quarter'] = evaluation_df['date'].dt.quarter
evaluation_df['month'] = evaluation_df['date'].dt.month
evaluation_df['week'] = evaluation_df['date'].dt.isocalendar().week.astype(np.int16)
evaluation_df['day_of_year'] = evaluation_df['date'].dt.dayofyear

validation_df['quarter'] = validation_df['date'].dt.quarter
validation_df['month'] = validation_df['date'].dt.month
validation_df['week'] = validation_df['date'].dt.isocalendar().week.astype(np.int16)
validation_df['day_of_year'] = validation_df['date'].dt.dayofyear


In [31]:
evaluation_df

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,quarter,week,day_of_year
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,1,1,2011,No Event,No Event,No Event,No Event,0,0,0,0.46,1,4,29
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,1,1,2011,No Event,No Event,No Event,No Event,0,0,0,0.46,1,4,29
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,1,1,2011,No Event,No Event,No Event,No Event,0,0,0,0.46,1,4,29
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,1,1,2011,No Event,No Event,No Event,No Event,0,0,0,0.46,1,4,29
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,1,1,2011,No Event,No Event,No Event,No Event,0,0,0,0.46,1,4,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60034805,FOODS_3_823_WI_3_evaluation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,d_1969,0,2016-06-19,11621,Sunday,2,6,2016,NBAFinalsEnd,Sporting,Father's day,Cultural,0,0,0,2.98,2,24,171
60034806,FOODS_3_824_WI_3_evaluation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,d_1969,0,2016-06-19,11621,Sunday,2,6,2016,NBAFinalsEnd,Sporting,Father's day,Cultural,0,0,0,2.48,2,24,171
60034807,FOODS_3_825_WI_3_evaluation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,d_1969,0,2016-06-19,11621,Sunday,2,6,2016,NBAFinalsEnd,Sporting,Father's day,Cultural,0,0,0,3.98,2,24,171
60034808,FOODS_3_826_WI_3_evaluation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,d_1969,0,2016-06-19,11621,Sunday,2,6,2016,NBAFinalsEnd,Sporting,Father's day,Cultural,0,0,0,1.28,2,24,171


In [32]:
validation_df

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,quarter,week,day_of_year
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,1,1,2011,No Event,No Event,No Event,No Event,0,0,0,0.46,1,4,29
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,1,1,2011,No Event,No Event,No Event,No Event,0,0,0,0.46,1,4,29
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,1,1,2011,No Event,No Event,No Event,No Event,0,0,0,0.46,1,4,29
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,1,1,2011,No Event,No Event,No Event,No Event,0,0,0,0.46,1,4,29
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,1,1,2011,No Event,No Event,No Event,No Event,0,0,0,0.46,1,4,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59181085,FOODS_3_823_WI_3_validation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,d_1941,0,2016-05-22,11617,Sunday,2,5,2016,No Event,No Event,No Event,No Event,0,0,0,2.98,2,20,143
59181086,FOODS_3_824_WI_3_validation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,d_1941,0,2016-05-22,11617,Sunday,2,5,2016,No Event,No Event,No Event,No Event,0,0,0,2.48,2,20,143
59181087,FOODS_3_825_WI_3_validation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,d_1941,0,2016-05-22,11617,Sunday,2,5,2016,No Event,No Event,No Event,No Event,0,0,0,3.98,2,20,143
59181088,FOODS_3_826_WI_3_validation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,d_1941,0,2016-05-22,11617,Sunday,2,5,2016,No Event,No Event,No Event,No Event,0,0,0,1.28,2,20,143


In [33]:
lags = [7, 14, 28, 35, 42, 56]
for lag in lags:
    evaluation_df[f'sales_lag_{lag}'] = evaluation_df.groupby(['store_id', 'item_id'])['sales'].shift(lag)
    validation_df[f'sales_lag_{lag}'] = validation_df.groupby(['store_id', 'item_id'])['sales'].shift(lag)


In [34]:
# ROLLING FEATURES (shifted to avoid data leakage)
window_sizes = [7, 14, 28, 35, 42, 56]
for window in window_sizes:
    evaluation_df[f'rolling_mean_{window}'] = (
        evaluation_df.groupby(['store_id', 'item_id'])['sales']
        .transform(lambda x: x.shift(1).rolling(window, min_periods=1).mean())
    )
    evaluation_df[f'rolling_std_{window}'] = (
        evaluation_df.groupby(['store_id', 'item_id'])['sales']
        .transform(lambda x: x.shift(1).rolling(window, min_periods=1).std())
    )
    
    validation_df[f'rolling_mean_{window}'] = (
        validation_df.groupby(['store_id', 'item_id'])['sales']
        .transform(lambda x: x.shift(1).rolling(window, min_periods=1).mean())
    )
    validation_df[f'rolling_std_{window}'] = (
        validation_df.groupby(['store_id', 'item_id'])['sales']
        .transform(lambda x: x.shift(1).rolling(window, min_periods=1).std())
    )


In [35]:
# PRICE-BASED FEATURES
evaluation_df['price_diff'] = (
    evaluation_df.groupby(['store_id', 'item_id'])['sell_price']
    .diff()
)
validation_df['price_diff'] = (
    validation_df.groupby(['store_id', 'item_id'])['sell_price']
    .diff()
)

evaluation_df['price_variance'] = (
    evaluation_df.groupby(['store_id', 'item_id'])['sell_price']
    .pct_change()
)
validation_df['price_variance'] = (
    validation_df.groupby(['store_id', 'item_id'])['sell_price']
    .pct_change()
)


In [36]:
evaluation_df['price_norm'] = (
    evaluation_df['sell_price'] / evaluation_df.groupby('store_id')['sell_price'].transform('mean')
)

validation_df['price_norm'] = (
    validation_df['sell_price'] / validation_df.groupby('store_id')['sell_price'].transform('mean')
)


In [37]:
# WEEKEND AND SNAP FLAGS
evaluation_df['wday'] = evaluation_df['wday'].astype(int)
validation_df['wday'] = validation_df['wday'].astype(int)

evaluation_df['is_weekend'] = evaluation_df['wday'].isin([1, 2]).astype(int)
validation_df['is_weekend'] = validation_df['wday'].isin([1, 2]).astype(int)

evaluation_df['is_snap'] = (evaluation_df[['snap_CA', 'snap_TX', 'snap_WI']].sum(axis=1) > 0).astype(int)
validation_df['is_snap'] = (validation_df[['snap_CA', 'snap_TX', 'snap_WI']].sum(axis=1) > 0).astype(int)


In [38]:
# Max historical price per store & item
price_max_eval = evaluation_df.groupby(['store_id', 'item_id'])['sell_price'].transform('max')
price_max_val = validation_df.groupby(['store_id', 'item_id'])['sell_price'].transform('max')

# Price momentum = current price / max price
evaluation_df['price_momentum'] = evaluation_df['sell_price'] / price_max_eval
validation_df['price_momentum'] = validation_df['sell_price'] / price_max_val


In [39]:
evaluation_df.dtypes

id                         object
item_id                    object
dept_id                    object
cat_id                     object
store_id                   object
state_id                   object
d                          object
sales                       int16
date               datetime64[ns]
wm_yr_wk                    int16
weekday                    object
wday                        int64
month                       int64
year                        int16
event_name_1               object
event_type_1               object
event_name_2               object
event_type_2               object
snap_CA                      int8
snap_TX                      int8
snap_WI                      int8
sell_price                float32
quarter                     int64
week                        int16
day_of_year                 int64
sales_lag_7               float64
sales_lag_14              float64
sales_lag_28              float64
sales_lag_35              float64
sales_lag_42  

In [40]:
# Columns to encode
cat_cols = [
    'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id',
    'weekday', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2'
]

# Convert to category then to integer codes
for col in cat_cols:
    evaluation_df[col] = evaluation_df[col].astype('category').cat.codes
    validation_df[col] = validation_df[col].astype('category').cat.codes


In [41]:
# Convert 'd' from 'd_1234' → 1234
validation_df['d'] = validation_df['d'].str.replace('d_', '').astype(int)
evaluation_df['d'] = evaluation_df['d'].str.replace('d_', '').astype(int)


In [5]:
pip install lightgbm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 36.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... one
  Preparing metadata (pyproject.toml) ... done
  Created wheel for lightgbm: filename=lightgbm-4.6.0-py3-none-linux_x86_64.whl size=2737778 sha256=0d579feee9ef7753e2122bfaff5cc959f935c2113e14abdc2764b62da2efd05a
  Stored in directory: /home/ec2-user/.cache/pip/wheels/bb/db/6d/7814aed03437129dc284a055c084f201b765deb54b6908efab
Successfully built lightgbm
Note: you may need to restart the kernel to use updated packages.


In [6]:
import lightgbm as lgb
import gc
from sklearn.metrics import mean_squared_error


In [7]:
# Define features (same as before)
features = [
    'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id',
    'wday', 'month', 'year', 'quarter', 'week', 'day_of_year',
    'weekday',
    'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2',
    'snap_CA', 'snap_TX', 'snap_WI',
    'sell_price', 'price_diff', 'price_variance', 'price_norm', 'price_momentum',
    'sales_lag_7', 'sales_lag_14', 'sales_lag_28', 'sales_lag_35',
    'sales_lag_42', 'sales_lag_56',
    'rolling_mean_7', 'rolling_std_7',
    'rolling_mean_14', 'rolling_std_14',
    'rolling_mean_28', 'rolling_std_28',
    'rolling_mean_35', 'rolling_std_35',
    'rolling_mean_42', 'rolling_std_42',
    'rolling_mean_56', 'rolling_std_56',
    'is_weekend', 'is_snap'
]

cat_features = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id',
                'weekday', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']


In [64]:
# Time-based CV splits
cv_splits = [(1830, 1858), (1858, 1886), (1886, 1914)]
cv_scores = []
submission_df = pd.DataFrame()
store_models = {}

# Unique stores
stores = validation_df['store_id'].unique()

for store in stores:
    print(f"📦 Training for store_id: {store}")
    store_df = validation_df[validation_df['store_id'] == store].copy()
    store_df['sales'] = store_df['sales'].astype(np.float32)

    for i, (train_start, valid_start) in enumerate(cv_splits):
        train = store_df[store_df['d'] < valid_start]
        valid = store_df[(store_df['d'] >= valid_start) & (store_df['d'] < valid_start + 28)]
        if train.empty or valid.empty:
            continue

        train_data = lgb.Dataset(train[features], label=np.log1p(train['sales']), categorical_feature=cat_features)
        valid_data = lgb.Dataset(valid[features], label=np.log1p(valid['sales']), categorical_feature=cat_features)

        params = {
            'boosting_type': 'gbdt',
            'objective': 'tweedie',
            'tweedie_variance_power': 1.1,
            'metric': 'rmse',
            'learning_rate': 0.02,
            'num_leaves': 2**8-1,
            'min_data_in_leaf': 100,
            'feature_fraction': 0.8,
            'max_bin': 100,
            'boost_from_average': False,
            'verbose': -1,
            'seed': 42
        }


        model = lgb.train(params, train_data, valid_sets=[train_data, valid_data],
                          num_boost_round=1500,
                          callbacks=[lgb.log_evaluation(100)])

        preds = np.expm1(model.predict(valid[features]))
        rmse = np.sqrt(mean_squared_error(valid['sales'], preds))
        cv_scores.append(rmse)
        print(f"✅ CV{i + 1} RMSE: {rmse:.4f}")

    # Final train on all before d_1914
    train_final = store_df[store_df['d'] < 1914]
    valid_public = store_df[(store_df['d'] >= 1914) & (store_df['d'] <= 1941)]

    final_data = lgb.Dataset(train_final[features], label=np.log1p(train_final['sales']), categorical_feature=cat_features)
    model = lgb.train(params, final_data, num_boost_round=1500, callbacks=[lgb.log_evaluation(100)])
    store_models[store] = model

    # Predict on public validation
    valid_public['sales'] = np.expm1(model.predict(valid_public[features]))
    submission_df = pd.concat([submission_df, valid_public[['id', 'd', 'sales']]])

    del train_final, valid_public, final_data, store_df, model
    gc.collect()

print("✅ Done training all stores!")
print(f"📊 CV Mean RMSE: {np.mean(cv_scores):.5f} | Std: {np.std(cv_scores):.5f}")

# Save predictions
submission_df.reset_index(drop=True, inplace=True)
submission_df.to_csv("validation_predictions.csv", index=False)


📦 Training for store_id: 0
[100]	training's rmse: 0.449497	valid_1's rmse: 0.493824
[200]	training's rmse: 0.435084	valid_1's rmse: 0.484344
[300]	training's rmse: 0.431801	valid_1's rmse: 0.483477
[400]	training's rmse: 0.42943	valid_1's rmse: 0.483125
[500]	training's rmse: 0.42696	valid_1's rmse: 0.482962
[600]	training's rmse: 0.424931	valid_1's rmse: 0.482935
[700]	training's rmse: 0.42313	valid_1's rmse: 0.482893
[800]	training's rmse: 0.421669	valid_1's rmse: 0.482913
[900]	training's rmse: 0.420367	valid_1's rmse: 0.482939
[1000]	training's rmse: 0.419173	valid_1's rmse: 0.482985
[1100]	training's rmse: 0.418145	valid_1's rmse: 0.483059
[1200]	training's rmse: 0.417078	valid_1's rmse: 0.483135
[1300]	training's rmse: 0.41612	valid_1's rmse: 0.483115
[1400]	training's rmse: 0.415111	valid_1's rmse: 0.483125
[1500]	training's rmse: 0.41426	valid_1's rmse: 0.483136
✅ CV1 RMSE: 2.0790
[100]	training's rmse: 0.45017	valid_1's rmse: 0.500313
[200]	training's rmse: 0.435812	valid_1's 

/tmp/ipykernel_11587/526199711.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_public['sales'] = np.expm1(model.predict(valid_public[features]))


📦 Training for store_id: 1
[100]	training's rmse: 0.421092	valid_1's rmse: 0.509732
[200]	training's rmse: 0.406623	valid_1's rmse: 0.502061
[300]	training's rmse: 0.403598	valid_1's rmse: 0.50258
[400]	training's rmse: 0.401468	valid_1's rmse: 0.502877
[500]	training's rmse: 0.399236	valid_1's rmse: 0.503222
[600]	training's rmse: 0.397294	valid_1's rmse: 0.503565
[700]	training's rmse: 0.395592	valid_1's rmse: 0.503896
[800]	training's rmse: 0.394227	valid_1's rmse: 0.504154
[900]	training's rmse: 0.392932	valid_1's rmse: 0.504286
[1000]	training's rmse: 0.391784	valid_1's rmse: 0.504464
[1100]	training's rmse: 0.390643	valid_1's rmse: 0.504703
[1200]	training's rmse: 0.389586	valid_1's rmse: 0.505017
[1300]	training's rmse: 0.38861	valid_1's rmse: 0.505349
[1400]	training's rmse: 0.387813	valid_1's rmse: 0.505682
[1500]	training's rmse: 0.386913	valid_1's rmse: 0.506247
✅ CV1 RMSE: 2.0341
[100]	training's rmse: 0.42252	valid_1's rmse: 0.511949
[200]	training's rmse: 0.408142	valid_1

/tmp/ipykernel_11587/526199711.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_public['sales'] = np.expm1(model.predict(valid_public[features]))


📦 Training for store_id: 2
[100]	training's rmse: 0.476397	valid_1's rmse: 0.536082
[200]	training's rmse: 0.460918	valid_1's rmse: 0.52732
[300]	training's rmse: 0.457605	valid_1's rmse: 0.526734
[400]	training's rmse: 0.455198	valid_1's rmse: 0.526657
[500]	training's rmse: 0.452811	valid_1's rmse: 0.526723
[600]	training's rmse: 0.45074	valid_1's rmse: 0.526784
[700]	training's rmse: 0.448949	valid_1's rmse: 0.526877
[800]	training's rmse: 0.447472	valid_1's rmse: 0.527003
[900]	training's rmse: 0.446146	valid_1's rmse: 0.527107
[1000]	training's rmse: 0.444921	valid_1's rmse: 0.52721
[1100]	training's rmse: 0.443812	valid_1's rmse: 0.527297
[1200]	training's rmse: 0.442664	valid_1's rmse: 0.527425
[1300]	training's rmse: 0.441687	valid_1's rmse: 0.527556
[1400]	training's rmse: 0.440657	valid_1's rmse: 0.527667
[1500]	training's rmse: 0.439696	valid_1's rmse: 0.527795
✅ CV1 RMSE: 2.7882
[100]	training's rmse: 0.477308	valid_1's rmse: 0.530645
[200]	training's rmse: 0.461903	valid_1

...

📦 Training for store_id: 8
[100]	training's rmse: 0.417729	valid_1's rmse: 0.495833
[200]	training's rmse: 0.401526	valid_1's rmse: 0.484418
[300]	training's rmse: 0.398301	valid_1's rmse: 0.483412
[400]	training's rmse: 0.396088	valid_1's rmse: 0.48331
[500]	training's rmse: 0.393439	valid_1's rmse: 0.483314
[600]	training's rmse: 0.391347	valid_1's rmse: 0.483528
[700]	training's rmse: 0.389652	valid_1's rmse: 0.483785
[800]	training's rmse: 0.388022	valid_1's rmse: 0.484085
[900]	training's rmse: 0.386655	valid_1's rmse: 0.48436
[1000]	training's rmse: 0.385367	valid_1's rmse: 0.484744
[1100]	training's rmse: 0.384175	valid_1's rmse: 0.485156
[1200]	training's rmse: 0.382989	valid_1's rmse: 0.485691
[1300]	training's rmse: 0.381933	valid_1's rmse: 0.486054
[1400]	training's rmse: 0.380998	valid_1's rmse: 0.486534
[1500]	training's rmse: 0.380073	valid_1's rmse: 0.486848
✅ CV1 RMSE: 3.0136
[100]	training's rmse: 0.418964	valid_1's rmse: 0.489718
[200]	training's rmse: 0.402813	valid_

/tmp/ipykernel_11587/526199711.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_public['sales'] = np.expm1(model.predict(valid_public[features]))


📦 Training for store_id: 9
[100]	training's rmse: 0.412342	valid_1's rmse: 0.458016
[200]	training's rmse: 0.396214	valid_1's rmse: 0.446486
[300]	training's rmse: 0.393069	valid_1's rmse: 0.445605
[400]	training's rmse: 0.39067	valid_1's rmse: 0.445403
[500]	training's rmse: 0.388367	valid_1's rmse: 0.445233
[600]	training's rmse: 0.386468	valid_1's rmse: 0.445203
[700]	training's rmse: 0.384717	valid_1's rmse: 0.445206
[800]	training's rmse: 0.38325	valid_1's rmse: 0.445205
[900]	training's rmse: 0.381819	valid_1's rmse: 0.4452
[1000]	training's rmse: 0.380546	valid_1's rmse: 0.445228
[1100]	training's rmse: 0.379401	valid_1's rmse: 0.445318
[1200]	training's rmse: 0.378339	valid_1's rmse: 0.44539
[1300]	training's rmse: 0.377417	valid_1's rmse: 0.445489
[1400]	training's rmse: 0.376503	valid_1's rmse: 0.44555
[1500]	training's rmse: 0.375711	valid_1's rmse: 0.445655
✅ CV1 RMSE: 2.0724
[100]	training's rmse: 0.413034	valid_1's rmse: 0.465634
[200]	training's rmse: 0.39694	valid_1's r

/tmp/ipykernel_11587/526199711.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_public['sales'] = np.expm1(model.predict(valid_public[features]))


✅ Done training all stores!
📊 CV Mean RMSE: 1.63552 | Std: 0.67440


In [51]:
import joblib

In [52]:
# Placeholder for all predictions
submission_eval = pd.DataFrame()

# Loop over each store
stores = evaluation_df['store_id'].unique()

for store in stores:
    print(f"📦 Training model for store_id: {store}")

    # Filter data for the store
    store_df = evaluation_df[evaluation_df['store_id'] == store].copy()

    # Split into train and forecast range
    train_df = store_df[store_df['d'] <= 1941]
    test_df = store_df[store_df['d'] > 1941].copy()

    # LightGBM dataset
    train_data = lgb.Dataset(train_df[features], label=np.log1p(train_df['sales']))

    # LightGBM parameters
    params = {
        'boosting_type': 'gbdt',
        'objective': 'tweedie',
        'tweedie_variance_power': 1.1,
        'metric': 'rmse',
        'learning_rate': 0.02,
        'num_leaves': 2**8 - 1,
        'min_data_in_leaf': 100,
        'feature_fraction': 0.8,
        'max_bin': 100,
        'boost_from_average': False,
        'verbose': -1,
        'seed': 42
    }

    # Train
    model = lgb.train(params, train_data, num_boost_round=1500, 
                      valid_sets=[train_data], 
                      callbacks=[lgb.log_evaluation(100)])

    # Predict on test (d_1942 to d_1969)
    test_df['sales'] = np.expm1(model.predict(test_df[features]))

    # Append to submission
    submission_eval = pd.concat([submission_eval, test_df[['id', 'd', 'sales']]])

    # Save model to file for batch pipeline
    model_path = f"models/lgbm_store_{store}.pkl"
    joblib.dump(model, model_path)
    print(f"✅ Model saved: {model_path}")

    # Cleanup
    del store_df, train_df, test_df, train_data, model
    gc.collect()

# Final message
print("\n✅ All store models trained and saved!")


📦 Training model for store_id: 0
[100]	training's rmse: 0.455668
[200]	training's rmse: 0.443869
[300]	training's rmse: 0.442367
[400]	training's rmse: 0.441343
[500]	training's rmse: 0.440459
[600]	training's rmse: 0.43971
[700]	training's rmse: 0.439128
[800]	training's rmse: 0.438614
[900]	training's rmse: 0.438131
[1000]	training's rmse: 0.437607
[1100]	training's rmse: 0.437134
[1200]	training's rmse: 0.436626
[1300]	training's rmse: 0.436151
[1400]	training's rmse: 0.435748
[1500]	training's rmse: 0.435326
✅ Model saved: models/lgbm_store_0.pkl
📦 Training model for store_id: 1
[100]	training's rmse: 0.429077
[200]	training's rmse: 0.417192
[300]	training's rmse: 0.415863
[400]	training's rmse: 0.415041
[500]	training's rmse: 0.414314
[600]	training's rmse: 0.413496
[700]	training's rmse: 0.412871
[800]	training's rmse: 0.412339
[900]	training's rmse: 0.411818
[1000]	training's rmse: 0.411348
[1100]	training's rmse: 0.410922
[1200]	training's rmse: 0.41047
[1300]	training's rmse: 

In [54]:
submission_eval.reset_index(drop=True, inplace=True)
submission_eval.to_csv("evaluation_predictions.csv", index=False)

In [45]:
import io

In [46]:
# S3 bucket and folder
bucket_name = "walmart-forecast-data"
prefix = "processed_data/"  # Folder inside the bucket

# Initialize S3 Client
s3 = boto3.client("s3")

# Function to save DataFrame to S3 as Parquet
def save_df_to_s3_parquet(df, file_name):
    parquet_buffer = io.BytesIO()
    df.to_parquet(parquet_buffer, index=False, engine="pyarrow")
    s3.put_object(Bucket=bucket_name, Key=prefix + file_name, Body=parquet_buffer.getvalue())
    print(f"✅ Saved `{file_name}` to S3!")

# Save validation and evaluation datasets as Parquet
save_df_to_s3_parquet(validation_df, "train_validation.parquet")
save_df_to_s3_parquet(evaluation_df, "train_evaluation.parquet")


✅ Saved `train_validation.parquet` to S3!
✅ Saved `train_evaluation.parquet` to S3!


In [56]:
import os

In [58]:
local_model_dir = "models/"  # Local directory where models are saved
model_prefix = "models/"  # S3 folder for models

# Upload all LightGBM model files
for model_file in os.listdir(local_model_dir):
    if model_file.endswith(".pkl"):
        local_path = os.path.join(local_model_dir, model_file)
        s3_key = model_prefix + model_file
        
        s3.upload_file(local_path, bucket_name, s3_key)
        print(f"✅ Uploaded {model_file} to s3://{bucket_name}/{s3_key}")


✅ Uploaded lgbm_store_6.pkl to s3://walmart-forecast-data/models/lgbm_store_6.pkl
✅ Uploaded lgbm_store_2.pkl to s3://walmart-forecast-data/models/lgbm_store_2.pkl
✅ Uploaded lgbm_store_8.pkl to s3://walmart-forecast-data/models/lgbm_store_8.pkl
✅ Uploaded lgbm_store_7.pkl to s3://walmart-forecast-data/models/lgbm_store_7.pkl
✅ Uploaded lgbm_store_4.pkl to s3://walmart-forecast-data/models/lgbm_store_4.pkl
✅ Uploaded lgbm_store_1.pkl to s3://walmart-forecast-data/models/lgbm_store_1.pkl
✅ Uploaded lgbm_store_5.pkl to s3://walmart-forecast-data/models/lgbm_store_5.pkl
✅ Uploaded lgbm_store_9.pkl to s3://walmart-forecast-data/models/lgbm_store_9.pkl
✅ Uploaded lgbm_store_3.pkl to s3://walmart-forecast-data/models/lgbm_store_3.pkl
✅ Uploaded lgbm_store_0.pkl to s3://walmart-forecast-data/models/lgbm_store_0.pkl


In [54]:
df = pd.read_csv("./evaluation_predictions.csv")
df

,id,d,sales
0,HOBBIES_1_001_CA_1_evaluation,1942,0.751502
1,HOBBIES_1_002_CA_1_evaluation,1942,0.132546
2,HOBBIES_1_003_CA_1_evaluation,1942,0.457382
3,HOBBIES_1_004_CA_1_evaluation,1942,1.206247
4,HOBBIES_1_005_CA_1_evaluation,1942,0.923479
...,...,...,...
853715,FOODS_3_823_WI_3_evaluation,1969,0.081390
853716,FOODS_3_824_WI_3_evaluation,1969,0.053329
853717,FOODS_3_825_WI_3_evaluation,1969,0.079297
853718,FOODS_3_826_WI_3_evaluation,1969,0.067499


In [55]:
from io import BytesIO

def read_s3_parquet(file_name):
    obj = s3.get_object(Bucket=bucket_name, Key='processed_data/' + file_name)
    return pd.read_parquet(BytesIO(obj["Body"].read()))

evaluation_train = read_s3_parquet('train_evaluation.parquet')

In [56]:
#d_1969 -> 2016-06-19
evaluation_train = evaluation_train[['d', 'date']]

In [57]:
evaluation_train = evaluation_train[evaluation_train['d'] > 1941]

In [58]:
# Ensure 'd' column in df matches the format in calendar_df (e.g., 'd_1942')
df['d'] = 'd_' + df['d'].astype(str)
evaluation_train['d'] = 'd_' + evaluation_train['d'].astype(str)

In [59]:
evaluation_train

,d,date
59181090,d_1942,2016-05-23
59181091,d_1942,2016-05-23
59181092,d_1942,2016-05-23
59181093,d_1942,2016-05-23
59181094,d_1942,2016-05-23
...,...,...
60034805,d_1969,2016-06-19
60034806,d_1969,2016-06-19
60034807,d_1969,2016-06-19
60034808,d_1969,2016-06-19


In [62]:
# Deduplicate to avoid cartesian join
date_map = evaluation_train[['d', 'date']].drop_duplicates()


In [63]:
df

,id,d,sales
0,HOBBIES_1_001_CA_1_evaluation,d_1942,0.751502
1,HOBBIES_1_002_CA_1_evaluation,d_1942,0.132546
2,HOBBIES_1_003_CA_1_evaluation,d_1942,0.457382
3,HOBBIES_1_004_CA_1_evaluation,d_1942,1.206247
4,HOBBIES_1_005_CA_1_evaluation,d_1942,0.923479
...,...,...,...
853715,FOODS_3_823_WI_3_evaluation,d_1969,0.081390
853716,FOODS_3_824_WI_3_evaluation,d_1969,0.053329
853717,FOODS_3_825_WI_3_evaluation,d_1969,0.079297
853718,FOODS_3_826_WI_3_evaluation,d_1969,0.067499


In [65]:
# Merge to get date
df = df.merge(date_map, on='d', how='left')

# Now split `id` into components (remove '_evaluation' if present)
id_split = df['id'].str.replace('_evaluation', '', regex=False).str.split('_', expand=True)
df['item_id'] = id_split[2]
df['dept_id'] = id_split[1]
df['cat_id'] = id_split[0]
df['store_id'] = id_split[4]
df['state_id'] = id_split[3]


In [66]:
df

,id,d,sales,date,item_id,dept_id,cat_id,store_id,state_id
0,HOBBIES_1_001_CA_1_evaluation,d_1942,0.751502,2016-05-23,001,1,HOBBIES,1,CA
1,HOBBIES_1_002_CA_1_evaluation,d_1942,0.132546,2016-05-23,002,1,HOBBIES,1,CA
2,HOBBIES_1_003_CA_1_evaluation,d_1942,0.457382,2016-05-23,003,1,HOBBIES,1,CA
3,HOBBIES_1_004_CA_1_evaluation,d_1942,1.206247,2016-05-23,004,1,HOBBIES,1,CA
4,HOBBIES_1_005_CA_1_evaluation,d_1942,0.923479,2016-05-23,005,1,HOBBIES,1,CA
...,...,...,...,...,...,...,...,...,...
853715,FOODS_3_823_WI_3_evaluation,d_1969,0.081390,2016-06-19,823,3,FOODS,3,WI
853716,FOODS_3_824_WI_3_evaluation,d_1969,0.053329,2016-06-19,824,3,FOODS,3,WI
853717,FOODS_3_825_WI_3_evaluation,d_1969,0.079297,2016-06-19,825,3,FOODS,3,WI
853718,FOODS_3_826_WI_3_evaluation,d_1969,0.067499,2016-06-19,826,3,FOODS,3,WI
